## Stabilization/DVR: step 1

In [1]:
import numpy as np
from numpy.linalg import eigvals
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

In [2]:
#
#  extend path by location of the dvr package
#
import sys
sys.path.append('../../Python_libs')
#sys.path.append('/home/thomas/Current_Work/Jolanta-by-dvr/Python_libs')
import dvr
from jolanta import Jolanta_3D

In [3]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [4]:
#
# Jolanata parameters a, b, c:
#
#    bound state:  -12.26336 eV
#    resonance: (3.279526396 - 0.2079713j)  eV
jparam=(0.028, 1.0, 0.028)

In [5]:
#
#  compute DVR of T and V
#  then show the density of states
#  in a potential + energy-levels plot
#
xmin=0
xmax=20      # grid from -xmax to xmax
thresh = 5   # maximum energy for plot
ppB = 15     # grid points per Bohr

nGrid=int((xmax-xmin)*ppB)
xs = dvr.DVRGrid(xmin, xmax, nGrid)
Ts = dvr.KineticEnergy(1, xmin, xmax, nGrid)
Vs=Jolanta_3D(xs, jparam)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
h=float(xmax) / (nGrid+1.0)
scale=400

plt.plot(xs,Vs*au2eV, '-', color="black")
for i in range(n_ene):
    plt.plot(xs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
#plt.ylim(0, 20)
plt.xlabel('$x$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.ylim(-8,8)
plt.show()

  1   -0.26351131 au =     -7.17051 eV
  2    0.06066341 au =      1.65074 eV
  3    0.11675239 au =      3.17699 eV
  4    0.17971292 au =      4.89024 eV
  5    0.29854286 au =      8.12377 eV


### Stabilization loop



In [12]:
L_min=20        # smallest grid extent (box size)
L_max=45       # largest grid extent (box size)
nEs_keep=30    # how many energies are kept

s_min=1/L_max**2
s_max=1/L_min**2


n_s=121  
slist=np.linspace(s_min, s_max, num=n_s, endpoint=True)
Llist=1/np.sqrt(slist)

run_data = np.zeros((n_s, nEs_keep))  # array used to collect the energies
dc_data  = np.zeros((n_s, nEs_keep))  # array used to collect the DC energies

do_DC=False

for i in range(n_s):
    xmax=1/np.sqrt(slist[i])
    xmin=0
    nGrid=int((xmax-xmin)*ppB)
    xs = dvr.DVRGrid(xmin, xmax, nGrid)
    Vs = Jolanta_3D(xs, jparam)
    Ts = dvr.KineticEnergy(1, xmin, xmax, nGrid)
    energy, wf= dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)
    run_data[i,:] = energy[0:nEs_keep]

    if do_DC:
        C = wf[:,1:]
        TC = np.matmul(Ts,C)
        Tprime = np.matmul(C.transpose(),TC)
        V0s=np.zeros(nGrid-1)  # no potential for DC states
        energy = dvr.DVRDiag2(nGrid, Tprime, V0s)
        dc_data[i,:] = energy[0:nEs_keep]
        
    print(i+1, end=" ")
    if (i+1)%10==0:
        print()

run_data *= au2eV
dc_data *= au2eV
slist*=1000

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 
111 112 113 114 115 116 117 118 119 120 
121 

<br> $E(L)$ and $E(\alpha)$ plots

In [18]:
plt.cla()
for i in range(0, nEs_keep):
    plt.plot(slist,run_data[:,i], 'o-', color='blue')
    if do_DC:
        plt.plot(slist,dc_data[:,i], '-', color='grey')
    plt.ylim(0,7)
plt.xscale('linear')
plt.show()

In [19]:
#
# copy the nEs lowest odd states from run_data to stab_data
#
nEs=6
stab_data = np.zeros((n_s,nEs+2))
i = 0
stab_data[:,0] = 1/np.sqrt(slist)
stab_data[:,1] = slist
for i in range(nEs):
    stab_data[:,i+2] = run_data[:,i]

# for checking what we got
plt.cla()
for i in range(nEs):
    plt.plot(stab_data[:,0],stab_data[:,i+2], '-', color='blue')
plt.ylim(0,10)
plt.show()

In [20]:
cols = ['L', 'z']
for i in range(nEs):
    cols.append('E'+str(i+1))
df = pd.DataFrame(stab_data, columns=cols)
df.head(5)

,L,z,E1,E2,E3,E4,E5,E6
0,1.423025,0.493827,-7.170508,0.159275,0.523245,1.098066,1.858843,2.728974
1,1.399532,0.510545,-7.170508,0.165750,0.545553,1.144416,1.933952,2.814184
2,1.377166,0.527263,-7.170508,0.172307,0.568155,1.191240,2.009268,2.890628
3,1.355838,0.543981,-7.170508,0.178946,0.591050,1.238525,2.084680,2.956664
4,1.335472,0.560700,-7.170508,0.185668,0.614238,1.286256,2.160057,3.011986


In [21]:
df.to_csv('DVR_stab_plot.csv', index=False)